# HDB Resale Price Prediction

## Importing Libraries

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
from bs4 import BeautifulSoup

## Importing Dataset

In [72]:
flat_data_2012_2014 = 'data/raw/flat_data( Mar 2012 to Dec 2014).csv'
flat_data_2015_2016 = 'data/raw/flat_data(Jan 2015 to Dec 2016).csv'
flat_data_2017_onwards = 'data/raw/flat_data(Jan-2017 onwards).csv'
mrt = 'data/raw/MRT.geojson'

flat_data_2012_2014_df = pd.read_csv(flat_data_2012_2014)
flat_data_2015_2016_df = pd.read_csv(flat_data_2015_2016)
flat_data_2017_onwards_df = pd.read_csv(flat_data_2017_onwards)
mrt_gdf = gpd.read_file(mrt)

### Fetching Location Data

In [73]:
def get_coordinates(address):
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": "Bearer **********************"}
    response = requests.get(url, headers=headers).json()

    if response['found'] > 0:
        lat = response['results'][0]['LATITUDE']
        lon = response['results'][0]['LONGITUDE']
        return lat, lon
    return None, None

# lat, lon = get_coordinates("Ang Mo Kio")

### Loading MRT Data

In [74]:
def extract_station_name(html_text):
    if pd.isna(html_text):
        return None
    soup = BeautifulSoup(html_text, "html.parser")

    station_row = soup.find("th", string="STATION_NA")
    if station_row:
        return station_row.find_next("td").text.strip()
    return None

mrt_gdf["station_name"] = mrt_gdf["Description"].apply(extract_station_name)
mrt_gdf["longitude"] = mrt_gdf.geometry.x
mrt_gdf["latitude"] = mrt_gdf.geometry.y

mrt_df = mrt_gdf[["station_name", "latitude", "longitude"]]

mrt_df.to_csv('data/clean_mrt_data.csv')

## Data Preprocessing

In [75]:
flat_data_2012_2014_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0


In [76]:
flat_data_2015_2016_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [77]:
flat_data_2017_onwards_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


### Data wrangling for "flat_data_2012_2014_df"

In [78]:
flat_data_2012_2014_df['month'] = pd.to_datetime(flat_data_2012_2014_df['month'], format='%Y-%m')
print(flat_data_2012_2014_df.dtypes)

month                  datetime64[ns]
town                           object
flat_type                      object
block                          object
street_name                    object
storey_range                   object
floor_area_sqm                float64
flat_model                     object
lease_commence_date             int64
resale_price                  float64
dtype: object


In [79]:
flat_data_2012_2014_df['remaining_lease'] =  (flat_data_2012_2014_df['lease_commence_date'] + 99)- flat_data_2012_2014_df['month'].dt.year

columns = flat_data_2012_2014_df.columns.to_list()
columns.remove('resale_price')
columns.append('resale_price')
flat_data_2012_2014_df = flat_data_2012_2014_df[columns]

flat_data_2012_2014_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,73,250000.0
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,67,265000.0
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,67,315000.0
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,71,320000.0
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,67,321000.0


In [80]:
flat_data_2012_2014_df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

### Data wrangling for "flat_data_2015_2016_df"

In [81]:
flat_data_2012_2014_df['month'] = pd.to_datetime(flat_data_2012_2014_df['month'], format='%Y-%m')
print(flat_data_2012_2014_df.dtypes)

month                  datetime64[ns]
town                           object
flat_type                      object
block                          object
street_name                    object
storey_range                   object
floor_area_sqm                float64
flat_model                     object
lease_commence_date             int64
remaining_lease                 int64
resale_price                  float64
dtype: object


In [82]:
flat_data_2012_2014_df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

### Data wrangling for "flat_data_2017_onwards_df"

In [83]:
flat_data_2017_onwards_df['month'] = pd.to_datetime(flat_data_2017_onwards_df['month'], format='%Y-%m')
print(flat_data_2017_onwards_df.dtypes)

month                  datetime64[ns]
town                           object
flat_type                      object
block                          object
street_name                    object
storey_range                   object
floor_area_sqm                float64
flat_model                     object
lease_commence_date             int64
remaining_lease                object
resale_price                  float64
dtype: object


In [84]:
flat_data_2017_onwards_df['remaining_lease'] = flat_data_2017_onwards_df['remaining_lease'].str[:2]
flat_data_2017_onwards_df['remaining_lease'] = flat_data_2017_onwards_df['remaining_lease'].astype('int')
flat_data_2017_onwards_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61,232000.0
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60,250000.0
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,262000.0
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62,265000.0
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,265000.0


In [85]:
flat_data_2012_2014_df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

### Combining all three DataFrame

In [89]:
hdb_df = pd.concat([flat_data_2012_2014_df, flat_data_2015_2016_df, flat_data_2017_onwards_df])
hdb_df['month'] = pd.to_datetime(hdb_df['month'])
hdb_df['town'] = hdb_df['town'].str.replace('KALLANG/WHAMPOA', 'KALLANG')
hdb_df = hdb_df.drop_duplicates()
hdb_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,73,250000.0
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,67,265000.0
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,67,315000.0
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,71,320000.0
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,67,321000.0


### Fetching Town Coordinates

In [90]:
unique_towns = hdb_df['town'].unique()
town_coordinates = {}

for town in unique_towns:
    town_coordinates[town] = get_coordinates(town)
    
hdb_df['latitude'] = hdb_df['town'].map(lambda x: town_coordinates[x][0])
hdb_df['longitude'] = hdb_df['town'].map(lambda x: town_coordinates[x][1])

columns = hdb_df.columns.to_list()
columns.remove('resale_price')
columns.append('resale_price')
hdb_df = hdb_df[columns]

### Finding each flat data with its nearest MRT Station

In [88]:
hdb_df = gpd.GeoDataFrame(hdb_df, geometry=gpd.points_from_xy(hdb_df["longitude"], hdb_df["latitude"]))
mrt_df = gpd.GeoDataFrame(mrt_df, geometry=gpd.points_from_xy(mrt_df["longitude"], mrt_df["latitude"]))

hdb_df = hdb_df.to_crs(epsg=3414)
mrt_df = mrt_df.to_crs(epsg=3414)

hdb_final_df = gpd.sjoin_nearest(hdb_df, mrt_df, how="left", distance_col="distance_km")

hdb_final_df = hdb_final_df[['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range','floor_area_sqm', 'flat_model', 'lease_commence_date', \
    'remaining_lease', 'latitude_left', 'longitude_left', 'station_name', 'distance_km', 'resale_price']]

ValueError: Cannot transform naive geometries.  Please set a crs on the object first.

In [ ]:
hdb_final_df = hdb_final_df.rename(columns={
    'latitude_left':'latitude',
    'longitude_left': 'longitude'
})

### Export Clean Dataset

In [ ]:
hdb_df.to_csv('data/clean_hdb_resale_data.csv')

## EDA